# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = os.path.join("..", "city_data.csv")
print(csv_path)
city_df = pd.read_csv(csv_path, encoding="utf-8")
city_df

../city_data.csv


,city,lat,lng,max temp,humidity,cloudiness,windspeed,country,date,name,max temp F
0,axim,4.8699,-2.2405,298.39,91.0,58.0,2.17,GH,1644453528,Esim,77.432
1,chuy,-33.6971,-53.4616,291.82,76.0,0.0,3.86,UY,1644453528,Chui,65.606
2,ushuaia,-54.8000,-68.3000,281.96,46.0,40.0,9.26,AR,1644453529,Ushuaia,47.858
3,mataura,-46.1927,168.8643,296.19,53.0,44.0,1.12,NZ,1644453529,Mataura,73.472
4,carnarvon,-24.8667,113.6333,300.19,69.0,0.0,3.09,AU,1644453529,Carnarvon,80.672
...,...,...,...,...,...,...,...,...,...,...,...
553,san jose,37.3394,-121.8950,296.91,30.0,75.0,5.14,US,1644453133,San Jose,74.768
554,atambua,-9.1061,124.8925,299.79,80.0,67.0,1.65,ID,1644453701,Atambua,79.952
555,kenda,23.2000,86.5333,289.68,77.0,100.0,0.93,IN,1644453702,Kenda,61.754
556,ouesso,1.6136,16.0517,295.64,33.0,81.0,1.64,CG,1644453702,Ouésso,72.482


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
weather_df = city_df.loc[(city_df['cloudiness'] == 0) & (city_df['windspeed'] < 10) & (city_df['max temp F'] < 80) & (city_df['max temp'] > 70)]
weather_df.count()

city          81
lat           81
lng           81
max temp      81
humidity      81
cloudiness    81
windspeed     81
country       80
date          81
name          81
max temp F    81
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
location_df = city_df[["lat", "lng"]]
weights = city_df["humidity"]
weights

0      91.0
1      76.0
2      46.0
3      53.0
4      69.0
       ... 
553    30.0
554    80.0
555    77.0
556    33.0
557    79.0
Name: humidity, Length: 558, dtype: float64

In [12]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location_df, weights=weights, dissipating=False, max_intensity=10, point_radius=1)
fig.add_layer(heat_layer)
plt.savefig("heatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = weather_df
hotel_df['hotel name'] = ""
hotel_df

/var/folders/4s/j3_l93ts54qgnyg57c9s590m0000gn/T/ipykernel_90346/403779039.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['hotel name'] = ""


,city,lat,lng,max temp,humidity,cloudiness,windspeed,country,date,name,max temp F,hotel name
1,chuy,-33.6971,-53.4616,291.82,76.0,0.0,3.86,UY,1644453528,Chui,65.606,
7,jamestown,42.0970,-79.2353,280.77,45.0,0.0,4.12,US,1644453501,Jamestown,45.716,
9,kapaa,22.0752,-159.3190,299.33,68.0,0.0,4.63,US,1644453530,Kapaa,79.124,
37,san quintin,30.4833,-115.9500,291.34,44.0,0.0,2.22,MX,1644453538,San Quintín,64.742,
39,arlit,18.7369,7.3853,291.91,9.0,0.0,4.24,NE,1644453539,Arlit,65.768,
...,...,...,...,...,...,...,...,...,...,...,...,...
528,tukrah,32.5341,20.5791,284.14,58.0,0.0,4.35,LY,1644453693,Tūkrah,51.782,
534,semirom,31.4167,51.5667,271.02,50.0,0.0,1.49,IR,1644453695,Semīrom,28.166,
539,camacha,33.0833,-16.3333,289.83,88.0,0.0,3.60,PT,1644453697,Camacha,62.024,
544,avila,40.5833,-5.0000,277.40,69.0,0.0,1.36,ES,1644453699,Avila,39.650,


In [8]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#target_search = ""
target_radius = 5000
target_type = ["hotel"]
lat = ""
lng = ""

In [9]:
from pprint import pprint

In [10]:
hotel_names = []
for index, row in hotel_df.iterrows():
    lat = row['lat']
    lng = row['lng']
    params = {
    "location": f"{lat},{lng}",
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    response = requests.get(url, params=params).json()

    try:
        hotel_names.append(response['results'][0]['name'])
    except:
        hotel_names.append("")
hotel_df["hotel name"]=hotel_names
hotel_df

/var/folders/4s/j3_l93ts54qgnyg57c9s590m0000gn/T/ipykernel_90346/973166775.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["hotel name"]=hotel_names


,city,lat,lng,max temp,humidity,cloudiness,windspeed,country,date,name,max temp F,hotel name
1,chuy,-33.6971,-53.4616,291.82,76.0,0.0,3.86,UY,1644453528,Chui,65.606,Chuy
7,jamestown,42.0970,-79.2353,280.77,45.0,0.0,4.12,US,1644453501,Jamestown,45.716,Jamestown
9,kapaa,22.0752,-159.3190,299.33,68.0,0.0,4.63,US,1644453530,Kapaa,79.124,Kapaʻa
37,san quintin,30.4833,-115.9500,291.34,44.0,0.0,2.22,MX,1644453538,San Quintín,64.742,Lázaro Cárdenas
39,arlit,18.7369,7.3853,291.91,9.0,0.0,4.24,NE,1644453539,Arlit,65.768,Arlit
...,...,...,...,...,...,...,...,...,...,...,...,...
528,tukrah,32.5341,20.5791,284.14,58.0,0.0,4.35,LY,1644453693,Tūkrah,51.782,Tocra
534,semirom,31.4167,51.5667,271.02,50.0,0.0,1.49,IR,1644453695,Semīrom,28.166,Semirom
539,camacha,33.0833,-16.3333,289.83,88.0,0.0,3.60,PT,1644453697,Camacha,62.024,Vila Baleira
544,avila,40.5833,-5.0000,277.40,69.0,0.0,1.36,ES,1644453699,Avila,39.650,Muñana


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [16]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("heatmp_with_markers.png")
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>